# NFL Big Data Bowl 2020 - My Solution

This notebook shows my own approach to the NFL Big Data Bowl 2020 competition. As the winner solution, our model is a convolutional network [1]. However, we exploit the concept of **_player influence area_** (Competition VIP Hint [2]) which was not taken into account for the majority of the winner solutions [3].

The **_player influence area_** is a concept introduced in [2] which characterized the influence that a player has over each point on the field as a function of 3 variables: player velocity, player position and ball position. The next figure illustrates this concept in a particular soccer play.

<img src="images/player_influence_area.gif" style="width:680px;height:340px;">

In this approach, we construct the input tensor to the convolutional network as an image around the rushing play, where each channel is the **_player influence area_** of each player. The next figure illustrates the idea behind the input tensor.

<img src="images/input_tensor.png" style="width:680px;height:240px;">

Similar to the notebook in [1], the remainder of this notebook is organized as follows. Section 1 contains the code for data processing and data augmentation. Section 2 provides the model structure. Finally, section 3 draws some conclusions and some possible improvements


## Data Processing

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import statsmodels.api as sm
import utils
from pitch_control import compute_player_influence
from tqdm import tqdm

In [4]:
train = pd.read_csv('data/train.csv', dtype={'WindSpeed': 'object'})

In [3]:
def split_play_and_player_cols(df, predicting=False):
    df['IsRusher'] = df['NflId'] == df['NflIdRusher']
    
    df['PlayId'] = df['PlayId'].astype(str)
    
    # We must assume here that the first 22 rows correspond to the same player:
    player_cols = [
        'PlayId', # This is the link between them
        'Season',
        'Team',
        'X',
        'Y',
        'S',
        'Dis',
        'Dir',
        'NflId',
        'IsRusher',
    ]

    df_players = df[player_cols]
    
    play_cols = [
        'PlayId',
        'Season',
        'PossessionTeam',
        'HomeTeamAbbr',
        'VisitorTeamAbbr',
        'PlayDirection', 
        'FieldPosition',
        'YardLine',
    ]
    
    if not predicting:
        play_cols.append('Yards')
        
    df_play = df[play_cols].copy()

    ## Fillna in FieldPosition attribute
    #df['FieldPosition'] = df.groupby(['PlayId'], sort=False)['FieldPosition'].apply(lambda x: x.ffill().bfill())
    
    # Get first 
    df_play = df_play.groupby('PlayId').first().reset_index()

    print('rows/plays in df: ', len(df_play))
    assert df_play.PlayId.nunique() == df.PlayId.nunique(), "Play/player split failed?"  # Boom
    
    return df_play, df_players

play_ids = train["PlayId"].unique()

df_play, df_players = split_play_and_player_cols(train)

rows/plays in df:  31007


In [4]:
def process_team_abbr(df):

    #These are only problems:
    map_abbr = {'ARI': 'ARZ', 'BAL': 'BLT', 'CLE': 'CLV', 'HOU': 'HST'}
    for abb in train['PossessionTeam'].unique():
        map_abbr[abb] = abb

    df['PossessionTeam'] = df['PossessionTeam'].map(map_abbr)
    df['HomeTeamAbbr'] = df['HomeTeamAbbr'].map(map_abbr)
    df['VisitorTeamAbbr'] = df['VisitorTeamAbbr'].map(map_abbr)

    df['HomePossession'] = df['PossessionTeam'] == df['HomeTeamAbbr']
    
    return

process_team_abbr(df_play)

In [5]:
def process_play_direction(df):
    df['IsPlayLeftToRight'] = df['PlayDirection'].apply(lambda val: True if val.strip() == 'right' else False)
    return

process_play_direction(df_play)

In [6]:
def process_yard_til_end_zone(df):
    def convert_to_yardline100(row):
        return (100 - row['YardLine']) if (row['PossessionTeam'] == row['FieldPosition']) else row['YardLine']
    df['Yardline100'] = df.apply(convert_to_yardline100, axis=1)
    return

process_yard_til_end_zone(df_play)

In [7]:
df_players = df_players.merge(
    df_play[['PlayId', 'PossessionTeam', 'HomeTeamAbbr', 'PlayDirection', 'Yardline100']], 
    how='left', on='PlayId')

In [8]:
df_players.loc[df_players.Season == 2017, 'S'] = 10*df_players.loc[df_players.Season == 2017,'Dis']

In [9]:
def standarize_direction(df):
    # adjusted the data to always be from left to right
    df['HomePossesion'] = df['PossessionTeam'] == df['HomeTeamAbbr']

    df['Dir_rad'] = np.mod(90 - df.Dir, 360) * math.pi/180.0

    df['ToLeft'] = df.PlayDirection == "left"
    df['TeamOnOffense'] = "home"
    df.loc[df.PossessionTeam != df.HomeTeamAbbr, 'TeamOnOffense'] = "away"
    df['IsOnOffense'] = df.Team == df.TeamOnOffense # Is player on offense?
    df['X_std'] = df.X
    df.loc[df.ToLeft, 'X_std'] = 120 - df.loc[df.ToLeft, 'X']
    df['Y_std'] = df.Y
    df.loc[df.ToLeft, 'Y_std'] = 160/3 - df.loc[df.ToLeft, 'Y']
    df['Dir_std'] = df.Dir_rad
    df.loc[df.ToLeft, 'Dir_std'] = np.mod(np.pi + df.loc[df.ToLeft, 'Dir_rad'], 2*np.pi)
   
    #Replace Null in Dir_rad
    df.loc[(df.IsOnOffense) & df['Dir_std'].isna(),'Dir_std'] = 0.0
    df.loc[~(df.IsOnOffense) & df['Dir_std'].isna(),'Dir_std'] = np.pi

standarize_direction(df_players)

### Data augmentation
For training, we assume that in a mirrored world the runs would have had the same outcomes. We apply 50% augmentation to flip the Y coordinates (and all respective relative features emerging from it). Furthermore, the function process_tracking_data computes the projections on X and Y for the velocity of each player and other features relative to rusher.

In [10]:
def data_augmentation(df, sample_ids):
    df_sample = df.loc[df.PlayId.isin(sample_ids)].copy()
    df_sample['Y_std'] = 160/3  - df_sample['Y_std']
    df_sample['Dir_std'] = df_sample['Dir_std'].apply(lambda x: 2*np.pi - x)
    df_sample['PlayId'] = df_sample['PlayId'].apply(lambda x: x+'_aug')
    return df_sample


sample_ids = np.random.choice(df_play.PlayId.unique(), int(0.5*len(df_play.PlayId.unique())))

df_players_aug = data_augmentation(df_players, sample_ids)
df_players = pd.concat([df_players, df_players_aug])
df_players.reset_index()

df_play_aug = df_play.loc[df_play.PlayId.isin(sample_ids)].copy()
df_play_aug['PlayId'] = df_play_aug['PlayId'].apply(lambda x: x+'_aug')
df_play = pd.concat([df_play, df_play_aug])
df_play.reset_index()

# This is necessary to maintain the order when in the next cell we use groupby
df_players.sort_values(by=['PlayId'],inplace=True)
df_play.sort_values(by=['PlayId'],inplace=True)

In [11]:
tracking_level_features = [
    'PlayId',
    'IsOnOffense',
    'X_std',
    'Y_std',
    'S',
    'Dir_std',
    'IsRusher'
]

df_all_feats = df_players[tracking_level_features]

print('Any null values: ', df_all_feats.isnull().sum().sum())

Any null values:  0


In [12]:
df_all_feats

,PlayId,IsOnOffense,X_std,Y_std,S,Dir_std,IsRusher
0,20170907000118,False,46.09,18.493333,4.00,1.620015,False
21,20170907000118,True,45.42,24.863333,2.40,0.250106,False
20,20170907000118,True,45.42,24.213333,2.20,0.487820,False
19,20170907000118,True,45.40,21.453333,1.70,0.046775,False
18,20170907000118,True,41.25,22.803333,3.80,0.423417,True
...,...,...,...,...,...,...,...
682150,20191125003789,False,47.84,28.243333,1.01,3.377561,False
682151,20191125003789,False,47.77,21.383333,1.75,3.137055,False
682152,20191125003789,False,47.92,26.593333,0.70,3.868697,False
682144,20191125003789,False,48.53,23.733333,0.71,4.232249,False


Finally, we create the train tensor to feed the convolutional network. The following image depicts the structure of the input tensor:

<img src="images/input_tensor.png" style="width:680px;height:240px;">

Note that the input image contain 22 channels, each channel represents the **_player influence area_** of a player


In [21]:
%%time 

grouped = df_all_feats.groupby('PlayId')
# channel first
train_x = np.zeros([len(grouped.size()),22,30,30])
i = 0
play_ids = df_play.PlayId.values
for name, group in tqdm(grouped):
    
    [[rusher_x, rusher_y, rusher_S, rusher_Dir]] = group.loc[group.IsRusher==1,['X_std', 'Y_std','S','Dir_std']].values
    ball_coords = [rusher_x, rusher_y]
    player_coords = [rusher_x, rusher_y]

    x = np.linspace(int(rusher_x), int(rusher_x) + 14, 30)
    y = np.linspace(17, 46, 30)
    X, Y = np.meshgrid(x, y)

    train_x[i,0] = compute_player_influence(X, Y, rusher_Dir, rusher_S, player_coords, ball_coords)

    offense_ids = group[group.IsOnOffense & ~group.IsRusher].index
    defense_ids = group[~group.IsOnOffense].index

    for j, offense_id in enumerate(offense_ids):
        [player_x, player_y, player_S, player_Dir] = group.loc[offense_id,['X_std', 'Y_std','S','Dir_std']].values
        player_coords = [player_x, player_y]
        train_x[i,j+1] = compute_player_influence(X, Y, player_Dir, player_S, player_coords, ball_coords)
    
    for j, defense_id in enumerate(defense_ids):
        [player_x, player_y, player_S, player_Dir] = group.loc[defense_id,['X_std', 'Y_std','S','Dir_std']].values
        player_coords = [player_x, player_y]
        train_x[i,j+11] = compute_player_influence(X, Y, player_Dir, player_S, player_coords, ball_coords)
    i+=1

np.save('data_pitch_control/train_x(augmented-50).npy', train_x)

100%|██████████| 43214/43214 [6:50:26<00:00,  1.75it/s]
CPU times: user 12h 25s, sys: 1h 24min 53s, total: 13h 25min 18s
Wall time: 6h 50min 56s


In [22]:
# Transform Y into indexed-classes:
train_y = df_play[['PlayId', 'Yards']].copy()

train_y['YardIndex'] = train_y['Yards'].apply(lambda val: val + 99)

min_idx_y = 71
max_idx_y = 150

train_y['YardIndexClipped'] = train_y['YardIndex'].apply(
    lambda val: min_idx_y if val < min_idx_y else max_idx_y if val > max_idx_y else val)

print('max yardIndex: ', train_y.YardIndex.max())
print('max yardIndexClipped: ', train_y.YardIndexClipped.max())
print('min yardIndex: ', train_y.YardIndex.min())
print('min yardIndexClipped: ', train_y.YardIndexClipped.min())

train_y.to_pickle('data_pitch_control/train_y.pkl')

max yardIndex:  198
max yardIndexClipped:  150
min yardIndex:  84
min yardIndexClipped:  84


In [23]:
df_season = df_play[['PlayId', 'Season']].copy()
df_season.to_pickle('data_pitch_control/df_season.pkl')

## Model Structure (ConvNet)

In [2]:
train_x = np.load('data_pitch_control/train_x(augmented-50).npy') 
train_y = pd.read_pickle('data_pitch_control/train_y.pkl') 
df_season = pd.read_pickle('data_pitch_control/df_season.pkl')

#num_classes_y = 199
min_idx_y = 71
max_idx_y = 150
num_classes_y = max_idx_y - min_idx_y + 1

# Define deffense values as negatives
train_x[:,11:] = -train_x[:,11:]

# Transpose to set the input as "channel_last"
train_x = np.transpose(train_x, (0,2,3,1))
train_x.shape

(43214, 30, 30, 22)

In [3]:
from tensorflow.keras.models import Model, Sequential

from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, AvgPool2D, Flatten,
    Input, BatchNormalization, Dense, Add, Lambda, Dropout, LayerNormalization)

from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import Callback, EarlyStopping

import tensorflow as tf 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

def crps(y_true, y_pred):
    loss = K.mean(K.sum((K.cumsum(y_pred, axis = 1) - K.cumsum(y_true, axis=1))**2, axis=1))/199
    return loss

The network architecture proposed in this approach follows the idea of classic convolutional networks (Le-Net, AlexNet or VGG-16).

In [4]:
def get_conv_net(num_classes_y):
    #_, x, y, z = train_x.shape
    model = Sequential()

    # Convolutional Layera
    model.add(Conv2D(32, input_shape=(30,30,22), kernel_size=(1,1), strides=(1,1), padding="same", activation = "relu"))
    model.add(Conv2D(32, kernel_size=(3,3), strides=(1,1), padding="same", activation = "relu"))

    # Max Pooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="same"))

    # Convolutional Layer
    model.add(Conv2D(64, kernel_size=(3,3), strides=(1,1), padding="valid", activation = "relu"))
    
    # Max Pooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="valid"))

    # Convolutional Layer
    model.add(Conv2D(64, kernel_size=(3,3), strides=(1,1), padding="valid", activation = "relu"))
    
    # Max Pooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="valid"))

    # Passing it to a Fully Connected layer
    model.add(Flatten())
    # 1st Fully Connected Layer
    model.add(Dense(256, activation = "relu"))
    model.add(BatchNormalization())

    # 2nd Fully Connected Layer
    model.add(Dense(128, activation = "relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    # Output Layer
    model.add(Dense(num_classes_y, activation='softmax')) 
    return model

model = get_conv_net(num_classes_y)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        736       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 64)          36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 2, 2, 64)          0

In [5]:
class Metric(Callback):
    def __init__(self, model, callbacks, data):
        super().__init__()
        self.model = model
        self.callbacks = callbacks
        self.data = data

    def on_train_begin(self, logs=None):
        for callback in self.callbacks:
            callback.on_train_begin(logs)

    def on_train_end(self, logs=None):
        for callback in self.callbacks:
            callback.on_train_end(logs)

    def on_epoch_end(self, batch, logs=None):
        X_valid, y_valid = self.data[0], self.data[1]

        y_pred = self.model.predict(X_valid)
        y_true = np.clip(np.cumsum(y_valid, axis=1), 0, 1)
        y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1)
        val_s = ((y_true - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * X_valid.shape[0])
        logs['val_CRPS'] = val_s
        
        for callback in self.callbacks:
            callback.on_epoch_end(batch, logs)

In [13]:
%%time 

models = []
kf = KFold(n_splits=8, shuffle=True, random_state=42)
score = []

for i, (tdx, vdx) in enumerate(kf.split(train_x, train_y)):
    print(f'Fold : {i}')
    X_train, X_val = train_x[tdx], train_x[vdx],
    y_train, y_val = train_y.iloc[tdx]['YardIndexClipped'].values, train_y.iloc[vdx]['YardIndexClipped'].values
    season_val = df_season.iloc[vdx]['Season'].values

    y_train_values = np.zeros((len(y_train), num_classes_y), np.int32)
    for irow, row in enumerate(y_train):
        y_train_values[(irow, row - min_idx_y)] = 1
        
    y_val_values = np.zeros((len(y_val), num_classes_y), np.int32)
    for irow, row in enumerate(y_val - min_idx_y):
        y_val_values[(irow, row)] = 1

    val_idx = np.where(season_val!=2017)
    
    X_val = X_val[val_idx]
    y_val_values = y_val_values[val_idx]

    y_train_values = y_train_values.astype('float32')
    y_val_values = y_val_values.astype('float32')
    
    model = get_conv_net(num_classes_y)

    es = EarlyStopping(monitor='val_CRPS',
                        mode='min',
                        restore_best_weights=True,
                        verbose=0,
                        patience=10)
    
    es.set_model(model)
    metric = Metric(model, [es], [X_val, y_val_values])

    n_epochs = 30 
    
    opt = Adam(learning_rate=1e-3)
    model.compile(loss=crps,
                  optimizer=opt)
    
    model.fit(X_train,
              y_train_values, 
              epochs=n_epochs,
              batch_size=64,
              verbose=1,
              callbacks=[metric],
              validation_data=(X_val, y_val_values))

    val_crps_score = min(model.history.history['val_CRPS'])
    print("Val loss: {}".format(val_crps_score))
    
    score.append(val_crps_score)

    models.append(model)
    
print(np.mean(score))

Fold : 0
Train on 37812 samples, validate on 3270 samples
Epoch 1/20
37812/37812 [==============================] - 59s 2ms/sample - loss: 0.0159 - val_loss: 0.0141
Epoch 2/20
37812/37812 [==============================] - 52s 1ms/sample - loss: 0.0141 - val_loss: 0.0139
Epoch 3/20
37812/37812 [==============================] - 51s 1ms/sample - loss: 0.0137 - val_loss: 0.0137
Epoch 4/20
37812/37812 [==============================] - 51s 1ms/sample - loss: 0.0134 - val_loss: 0.0154
Epoch 5/20
37812/37812 [==============================] - 52s 1ms/sample - loss: 0.0132 - val_loss: 0.0137
Epoch 6/20
37812/37812 [==============================] - 53s 1ms/sample - loss: 0.0131 - val_loss: 0.0133
Epoch 7/20
37812/37812 [==============================] - 55s 1ms/sample - loss: 0.0130 - val_loss: 0.0131
Epoch 8/20
37812/37812 [==============================] - 55s 1ms/sample - loss: 0.0129 - val_loss: 0.0134
Epoch 9/20
37812/37812 [==============================] - 53s 1ms/sample - loss: 0.012

In [ ]:
print("The mean validation loss is {}".format(np.mean(score)))

## Conclusions

This approach is not possible to be submitted in kaggle because it does not satisfy the code requirements (CPU time less than 4 hours). Furthermore, it can be seen that we are adding complexity to the problem since we are converting tabular data as images. 

However, this first attempt shows that the player influence area concept is useful to predict the outcome of a rushing play, since we should reach the 71st place in the competition according to the CV score. Also, it can be improved in several ways:
- Improve the convolutional network architecture. The architecture proposes in this notebook has not been tunned.
- Avoid some unnecessary calculus in the player influence area. We are computing the player influence area for some points that are far away from the player and the result can be approximated as zero.
- Use a speed max based on the NFL data. The speed max is a parameter used to compute the player influence area. In this first attempt, we use a soccer speed max

With this first attempt, we aim to encourage other competitors to exploit the concept of the **_player influence area_** to predict the outcome of a rushing play.

[1] https://github.com/juancamilocampos/nfl-big-data-bowl-2020/blob/master/1st_place_zoo_solution_v2.ipynb

[2] Fernandez Javier, and Bornn Luke (2018). Wide Open Spaces: A statistical technique for measuring space creation in professional soccer.

[3] https://www.kaggle.com/c/nfl-big-data-bowl-2020/discussion/125010